<a href="https://colab.research.google.com/github/ZKhan2005/langchain-RAG--Pro/blob/main/LangChain_RAG_Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.2 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get("PINECONE_API_KEY")


pc = Pinecone(api_key=pinecone_api_key)

In [3]:
import time

index_name = "langchain-rag-pro"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]


pc.create_index(
name=index_name,
dimension=768,
metric="cosine",
spec=ServerlessSpec(cloud="aws", region="us-east-1"),
 )


index = pc.Index(index_name)

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
os.environ['GOOGLE_API_KEY'] =  userdata.get("GOOGLE_API_KEY")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [6]:
vector = embeddings.embed_query("I am building a RAG Text")

In [7]:
vector[:5]

[0.019773654639720917,
 -0.07713881880044937,
 -0.04218349978327751,
 -0.010219748131930828,
 0.03234664723277092]

In [8]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [9]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)


['9bd4e118-b350-48e1-a50a-09f038929e7b',
 '6d5e4e2a-cd42-4139-a08b-67998f1a322a',
 '43dd5795-0784-4880-8850-e6d060b7c941',
 'e4a38f1a-4bea-4932-88e4-6ef21a5ba54f',
 'f11e4004-5221-46ab-84c5-86a21a1dcfe5',
 '3dae9a5b-cc78-443b-9078-dfd3e0c03eb4',
 'c0d14464-a2e6-4b1c-aeae-598b18f68e06',
 '4d22bfc9-35c4-48b1-9df7-b4864452e1ce',
 'e39b5f87-cac0-4746-b522-505b4425e498',
 'efdadfd3-609e-4311-9120-aa3716d19b0e']

In [10]:
len(documents)

10

In [11]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['5e228154-778b-4122-9fd3-1598215e7288',
 '96f3a994-f8ba-4c68-aa35-4892978f0866',
 '8b6429ac-0ed3-4f0c-ad3a-61939c8722cf',
 'd8166dcc-5c7d-4130-a327-b36a29e636fd',
 'e06b7bd1-6421-4578-9687-f743d9ab60a9',
 'b783a9bf-948b-4226-b64c-3b8d2f6bf199',
 '4ebfc956-b01d-40f1-a7a2-6de57eaeee95',
 '9b9af5c7-a1f4-4796-a3b0-896aa85e3246',
 '22f74443-6a6a-4e6b-b85a-55ff4eaee60f',
 'df5f821e-bb64-4624-b3cf-bb845ca13726']

In [12]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [14]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?"
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.577411] I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]
* [SIM=0.577411] I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]


In [15]:
def answer_to_user(query: str) -> str:
  #vector search
    results = vector_store.similarity_search(query, k=2)
    # Pass the Model Vector  Results + User Query
    final_answer =model(results, query)
    return final_answer